In [1]:
import os, sys

import numpy as np
import pandas as pd 
from datasets import load_dataset

import importlib
from tqdm import tqdm
from joblib import Parallel, delayed
from copy import copy

from transformers import (
    AutoConfig,
    AutoTokenizer,
    FlaxAutoModelForSequenceClassification,
    HfArgumentParser,
    PretrainedConfig,
    TrainingArguments,
    is_tensorboard_available,
)

from flax.training.common_utils import get_metrics, onehot, shard


data_root = "/kaggle/input/feedback-prize-effectiveness/"
train = pd.read_csv("/kaggle/input/feedback-prize-effectiveness/train.csv")


/kaggle/working/jax/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LABEL_MAPPING = {"Ineffective": 0, "Adequate": 1, "Effective": 2}

def _prepare_training_data_helper(args, tokenizer, df, is_train):
    training_samples = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        idx = row["essay_id"]
        discourse_text = row["discourse_text"]
        discourse_type = row["discourse_type"]

        if is_train:
            filename = os.path.join(args.input, "train", idx + ".txt")
        else:
            filename = os.path.join(args.input, "test", idx + ".txt")

        with open(filename, "r") as f:
            text = f.read()

        encoded_text = tokenizer.encode_plus(
            discourse_type + " " + discourse_text,
            text,
            add_special_tokens=False,
            padding="max_length",
            truncation=True,
            max_length=1024 ##TODO: update max_length
        )
        input_ids = encoded_text["input_ids"]

        sample = {
            # "discourse_id": row["discourse_id"],
            "input_ids": input_ids,
            # "discourse_text": discourse_text,
            # "essay_text": text,
            "attention_mask": encoded_text["attention_mask"],
        }

        if "token_type_ids" in encoded_text:
            sample["token_type_ids"] = encoded_text["token_type_ids"]

        try:
            label = row["discourse_effectiveness"]
            sample["labels"] = LABEL_MAPPING[label]
        except:
            sample["labels"] = 0
        

        training_samples.append(sample)
    return training_samples


def prepare_training_data(df, tokenizer, args, num_jobs, is_train):
    training_samples = []

    df_splits = np.array_split(df, num_jobs)

    results = Parallel(n_jobs=num_jobs, backend="multiprocessing")(
        delayed(_prepare_training_data_helper)(args, tokenizer, df, is_train) for df in df_splits
    )
    for result in results:
        training_samples.extend(result)

    return training_samples

In [3]:
sys.path.append("../configs")
cfg = copy(importlib.import_module("elu_config").cfg)

print(cfg.model_name_or_path)

google/bigbird-roberta-large


In [4]:


# Load pretrained model and tokenizer
config = AutoConfig.from_pretrained(
    cfg.model_name_or_path,
    num_labels=cfg.num_labels,
    #finetuning_task=data_args.task_name,
    #use_auth_token=True if cfg.use_auth_token else None,
)
tokenizer = AutoTokenizer.from_pretrained(
    cfg.model_name_or_path,
    use_fast=not cfg.use_slow_tokenizer,
    #use_auth_token=True if cfg.use_auth_token else None,
)
model = FlaxAutoModelForSequenceClassification.from_pretrained(
    # cfg.model_name_or_path,
    cfg.model_name_or_path,
    config=config,
    ignore_mismatched_sizes=True,
    #use_auth_token=True if cfg.use_auth_token else None,
)

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
Some weights of the model checkpoint at google/bigbird-roberta-large were not used when initializing FlaxBigBirdForSequenceClassification: {('cls', 'predictions', 'transform', 'dense', 'bias'), ('cls', 'predictions', 'transform', 'LayerNorm', 'scale'), ('cls', 'seq_relationship', 'kernel'), ('cls', 'predictions', 'transform', 'LayerNorm', 'bias'), ('cls', 'predictions', 'bias'), ('cls', 'seq_relationship', 'bias'), ('cls', 'predictions', 'transform', 'dense', 'kernel')}
- This IS expected if you are initializing FlaxBigBirdForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxBigBirdForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSeque

In [5]:
cfg.model_name_or_path

'google/bigbird-roberta-large'

In [6]:
import json
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
## stratified Kfold for train dataframe using discourse_type and discourse_effectiveness
for fold, (train_index, valid_index) in enumerate(kf.split(train, train["discourse_type"], train["discourse_effectiveness"])):

    train_temp = train.iloc[train_index]
    valid_temp = train.iloc[valid_index]

    train_data = prepare_training_data(train_temp, tokenizer, cfg, num_jobs=96, is_train=True)
    val_data = prepare_training_data(valid_temp, tokenizer, cfg, num_jobs=96, is_train=True)

    df = pd.DataFrame.from_records(train_data)
    df.to_json(f"/kaggle/working/folds/train_{fold}.jsonl", orient="records", lines=True)

    df = pd.DataFrame.from_records(val_data)
    df.to_json(f"/kaggle/working/folds/valid_{fold}.jsonl", orient="records", lines=True)

    print("Fold:", fold)
    print("Train:", train_index)
    print("Valid:", valid_index)
    print("\n")

100%|██████████| 306/306 [00:00<00:00, 417.56it/s]
https://symbolize.stripped_domain/r/?trace=7f15e4ac73f4,7f15e4b1b0bf,7f&map= 
*** SIGTERM received by PID 663689 (TID 663689) on cpu 0 from PID 661991; stack trace: ***
PC: @     0x7f15e4ac73f4  (unknown)  do_futex_wait.constprop.0
    @     0x7f1462607c73        992  (unknown)
    @     0x7f15e4b1b0c0  (unknown)  (unknown)
    @               0x80  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f15e4ac73f4,7f1462607c72,7f15e4b1b0bf,7f&map=abc33f1bfca16f4e7d925d4248b4beb3:7f144ded9000-7f1462988b70 
E0621 06:33:26.734794  663689 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
https://symbolize.stripped_domain/r/?trace=7f1462664cc3,7f15e4b1b0bf,7f1462513666,7f146261d92a,7f14626234cc,7f1462622072,7f1462621b29,7f1462981d0d,7f146260870a,7f15e4b1b0bf,7f&map=abc33f1bfca16f4e7d925d4248b4beb3:7f144ded9000-7f1462988b70 
E0621 06:33:26.736804  663689 process_state.cc:1067] RAW: Signal 11 raised at PC: 0x7

Fold: 0
Train: [    0     1     4 ... 36762 36763 36764]
Valid: [    2     3     7 ... 36733 36736 36746]




100%|██████████| 306/306 [00:00<00:00, 427.79it/s]
https://symbolize.stripped_domain/r/?trace=7f15e4ac73f4,7f15e4b1b0bf,7f&map= 
*** SIGTERM received by PID 685099 (TID 685099) on cpu 50 from PID 661991; stack trace: ***
PC: @     0x7f15e4ac73f4  (unknown)  do_futex_wait.constprop.0
    @     0x7f1462607c73        992  (unknown)
    @     0x7f15e4b1b0c0  (unknown)  (unknown)
    @               0x80  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f15e4ac73f4,7f1462607c72,7f15e4b1b0bf,7f&map=abc33f1bfca16f4e7d925d4248b4beb3:7f144ded9000-7f1462988b70 
E0621 06:34:22.762964  685099 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
https://symbolize.stripped_domain/r/?trace=7f1462664cc3,7f15e4b1b0bf,7f1462513666,7f146261d92a,7f14626234cc,7f1462622072,7f1462621b29,7f1462981d0d,7f146260870a,7f15e4b1b0bf,7f&map=abc33f1bfca16f4e7d925d4248b4beb3:7f144ded9000-7f1462988b70 
E0621 06:34:22.765258  685099 process_state.cc:1067] RAW: Signal 11 raised at PC: 0x

Fold: 1
Train: [    0     2     3 ... 36759 36763 36764]
Valid: [    1     4     8 ... 36760 36761 36762]




100%|██████████| 306/306 [00:00<00:00, 411.53it/s]
https://symbolize.stripped_domain/r/?trace=7f15e4ac73f4,7f15e4b1b0bf,7f&map= 
*** SIGTERM received by PID 706377 (TID 706377) on cpu 16 from PID 661991; stack trace: ***
PC: @     0x7f15e4ac73f4  (unknown)  do_futex_wait.constprop.0
    @     0x7f1462607c73        992  (unknown)
    @     0x7f15e4b1b0c0  (unknown)  (unknown)
    @               0x80  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f15e4ac73f4,7f1462607c72,7f15e4b1b0bf,7f&map=abc33f1bfca16f4e7d925d4248b4beb3:7f144ded9000-7f1462988b70 
E0621 06:35:21.245178  706377 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
https://symbolize.stripped_domain/r/?trace=7f1462664cc3,7f15e4b1b0bf,7f1462513666,7f146261d92a,7f14626234cc,7f1462622072,7f1462621b29,7f1462981d0d,7f146260870a,7f15e4b1b0bf,7f&map=abc33f1bfca16f4e7d925d4248b4beb3:7f144ded9000-7f1462988b70 
E0621 06:35:21.247148  706377 process_state.cc:1067] RAW: Signal 11 raised at PC: 0x

Fold: 2
Train: [    0     1     2 ... 36761 36762 36763]
Valid: [    5     6     9 ... 36755 36759 36764]




100%|██████████| 307/307 [00:00<00:00, 581.35it/s]

100%|██████████| 76/76 [00:00<00:00, 375.82it/s]
https://symbolize.stripped_domain/r/?trace=7f15e4ac73f4,7f15e4b1b0bf,7f&map= 
*** SIGTERM received by PID 738812 (TID 738812) on cpu 71 from PID 661991; stack trace: ***
PC: @     0x7f15e4ac73f4  (unknown)  do_futex_wait.constprop.0
    @     0x7f1462607c73        992  (unknown)
    @     0x7f15e4b1b0c0  (unknown)  (unknown)
    @               0x80  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f15e4ac73f4,7f1462607c72,7f15e4b1b0bf,7f&map=abc33f1bfca16f4e7d925d4248b4beb3:7f144ded9000-7f1462988b70 
E0621 06:36:43.659145  738812 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
https://symbolize.stripped_domain/r/?trace=7f1462664cc3,7f15e4b1b0bf,7f1462513666,7f146261d92a,7f14626234cc,7f1462622072,7f1462621b29,7f1462981d0d,7f146260870a,7f15e4b1b0bf,7f&map=abc33f1bfca16f4e7d925d4248b4beb3:7f144ded9000-7f1462988b70 
E0621 06:36:43.661079  738812 proc

Fold: 3
Train: [    0     1     2 ... 36761 36762 36764]
Valid: [   10    22    23 ... 36745 36756 36763]




100%|██████████| 306/306 [00:00<00:00, 413.06it/s]
https://symbolize.stripped_domain/r/?trace=https://symbolize.stripped_domain/r/?trace=7f15e4ac7a65,7f15e4b1b0bf&map= 
*** SIGTERM received by PID 749042 (TID 749042) on cpu 71 from PID 661991; stack trace: ***
7f15e4ac73f4,7f15e4b1b0bf,7f&map= 
*** SIGTERM received by PID 749043 (TID 749043) on cpu 15 from PID 661991; stack trace: ***
PC: @     0x7f15e4ac7a65  (unknown)  sem_post@@GLIBC_2.2.5
    @     0x7f1462607c73        992  (unknown)
    @     0x7f15e4b1b0c0  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f15e4ac7a65,7f1462607c72,7f15e4b1b0bf&map=PC: @     0x7f15e4ac73f4  (unknown)  do_futex_wait.constprop.0
abc33f1bfca16f4e7d925d4248b4beb3:7f144ded9000-7f1462988b70 
    @     0x7f1462607c73        992  (unknown)
E0621 06:37:17.082115  749042 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
    @     0x7f15e4b1b0c0  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f1462664cc

Fold: 4
Train: [    1     2     3 ... 36762 36763 36764]
Valid: [    0    13    14 ... 36748 36753 36757]




In [7]:
## generate test dataset
test = pd.read_csv("/kaggle/input/feedback-prize-effectiveness/test.csv")
test_data = prepare_training_data(test, tokenizer, cfg, num_jobs=96, is_train=False)
df = pd.DataFrame.from_records(test_data)
df.to_json(f"/kaggle/working/folds/test.jsonl", orient="records", lines=True)

100%|██████████| 1/1 [00:00<00:00, 109.51it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]https://symbolize.stripped_domain/r/?trace=7ff144ae69ab,7ff144a110bf,7fe47842b9bf,7fe4788159bf,7ff1449c92df,7fe204f5a9bf,7fe1fec1e9bf,7fe1fee309bf,7fe1ff0429bf,7fe1ff2549bf,7fe1ff4669bf,7fe1ff6789bf,7fe1ff88a9bf,7fe2031219bf,7fe2033339bf,7fe2035459bf,7fe203b349bf,7fe203d469bf,7fe203f589bf,7fe2047599bf,7fe2037579bf,7fe20596d9bf,7fe205b7f9bf,7fe205d919bf,7fe205fa39bf,7fe2383d49bf,7fe2385e69bf,7fe2387f89bf,7fe2402b89bf,7fe2404ca9bf,7fea1a7fc9bf,7fe2406dc9bf,7fe2067a49bf&map= 
*** SIGTERM received by PID 3226283 (TID 3226283) on cpu 80 from PID 3117385; stack trace: ***
https://symbolize.stripped_domain/r/?trace=7ff1449bd3f4,7ff144a110bf,7f&map= 
*** SIGTERM receiv

KeyboardInterrupt: 

In [8]:
print("AFSAS")

AFSAS


In [11]:
## data collator with dynamic padding
# def train_data_collator(rng:)
import jax
import datasets
from typing import Any, Callable, Dict, Optional, Tuple

rng = jax.random.PRNGKey(1)#cfg.seed)
dropout_rngs = jax.random.split(rng, jax.local_device_count())

Array = Any
Dataset = datasets.arrow_dataset.Dataset
PRNGKey = Any

In [18]:
def eval_data_collator(dataset: Dataset, batch_size: int):
    """Returns batches of size `batch_size` from `eval dataset`, sharded over all local devices."""
    for i in range(len(dataset) // batch_size):
        batch = dataset[i * batch_size : (i + 1) * batch_size]
        batch = {k: np.array(v) for k, v in batch.items()}

        yield batch

In [19]:
rng = jax.random.PRNGKey(cfg.seed)
dropout_rngs = jax.random.split(rng, jax.local_device_count())
rng, input_rng = jax.random.split(rng)
eval_dataset   = load_dataset("json", data_files=f"/kaggle/working/folds/valid_{cfg.fold}.jsonl", split="train")
eval_loader = eval_data_collator(eval_dataset, 4)


In [20]:
for batch in eval_loader:
    print(batch['input_ids'].shape)    
    break

(4, 512)


In [22]:
labels = batch.pop("labels")
outs = model(**batch)

In [27]:
outs.logits, labels


(DeviceArray([[-0.5264392 ,  2.8834088 , -2.3014016 ],
              [ 0.8853584 ,  2.7341392 , -3.3088202 ],
              [ 0.59324175,  2.7294536 , -3.0007594 ],
              [-3.8339581 ,  0.4319707 ,  3.4270277 ]], dtype=float32),
 array([1, 1, 1, 2]))

In [13]:
## decode text using tokenizer
text = train_dataset[2]['input_ids']
text = tokenizer.decode(text)
print(text)

Counterclaim Everyone who thought it was made by alieans even though it wasn't, was not satisfied. I think they were not satisfied because they have thought since 1976 that it was really formed by alieans.  Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform. On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform<unk>I think that the face is a natural landform because there is no life on Mars that we have descovered yet. If life was on Mars, we would know by now. The reason why I think it is a natural landform because, nobody live on Mars in order to create the figure. It sa

In [12]:
len(np.nonzero(train_dataset[0]['input_ids'])[0])

512

In [14]:
train_dataset[0]['input_ids']

[22171,
 415,
 993,
 427,
 363,
 2087,
 419,
 358,
 3389,
 2057,
 788,
 881,
 713,
 419,
 746,
 1305,
 420,
 8807,
 427,
 457,
 524,
 849,
 32212,
 1966,
 321,
 16003,
 112,
 1413,
 1202,
 19169,
 112,
 1413,
 1202,
 1117,
 385,
 408,
 3698,
 647,
 804,
 529,
 2087,
 420,
 8807,
 419,
 358,
 3389,
 2057,
 788,
 494,
 712,
 713,
 419,
 1305,
 420,
 8807,
 427,
 1026,
 441,
 114,
 484,
 1722,
 419,
 647,
 804,
 8985,
 1819,
 358,
 4387,
 387,
 8807,
 391,
 358,
 2087,
 474,
 1876,
 420,
 363,
 5541,
 114,
 8985,
 1696,
 571,
 861,
 712,
 363,
 2057,
 788,
 474,
 2828,
 517,
 1305,
 420,
 8807,
 112,
 494,
 712,
 441,
 419,
 756,
 358,
 3389,
 2057,
 788,
 114,
 1651,
 717,
 6751,
 112,
 415,
 993,
 427,
 363,
 2087,
 419,
 358,
 3389,
 2057,
 788,
 881,
 415,
 17767,
 993,
 427,
 713,
 419,
 698,
 1305,
 420,
 8807,
 114,
 655,
 878,
 1407,
 1279,
 23650,
 112,
 415,
 1284,
 408,
 3476,
 647,
 804,
 415,
 993,
 427,
 419,
 419,
 358,
 3389,
 2057,
 788,
 100,
 141,
 993,
 427,
 363,
 208